In [1]:
from tensorflow.python.keras._impl.keras.preprocessing import sequence
from tensorflow.python.keras._impl.keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from tensorflow.python.keras._impl.keras.layers import Reshape, Flatten, Dropout, Concatenate, dot
from tensorflow.python.keras._impl.keras.optimizers import Adam
from tensorflow.python.keras._impl.keras.models import Model
from tensorflow.python.keras._impl.keras.layers import LSTM
from tensorflow.python import keras
from tensorflow.python.keras._impl.keras.layers.wrappers import TimeDistributed, Bidirectional

from tensorflow.python.keras._impl.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras._impl.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras._impl.keras.utils.data_utils import get_file

import tensorflow as tf
import tensorflow.contrib.eager as tfe
# tf.contrib.eager.enable_eager_execution()

import os
import numpy as np
import csv, json
from zipfile import ZipFile
from os.path import expanduser, exists
import pandas as pd

In [2]:
import os
from os.path import expanduser, exists

quora_path = expanduser('~/.kaggle/competitions/quora-question-pairs/')
quora_train = os.path.join(quora_path, 'train.csv')
quora_test = os.path.join(quora_path, 'test.csv')

In [3]:
# KERAS_DATASETS_DIRKERAS_D  = expanduser('~/.keras/datasets/')
# QUESTION_PAIRS_FILE_URL = 'http://qim.ec.quoracdn.net/quora_duplicate_questions.tsv'
# QUESTION_PAIRS_FILE = 'quora_duplicate_questions.tsv'
GLOVE_ZIP_FILE_URL = 'http://nlp.stanford.edu/data/glove.840B.300d.zip'
GLOVE_ZIP_FILE = 'glove.840B.300d.zip'
# GLOVE_FILE = 'glove.840B.300d.txt'
GLOVE_FILE = './data/glove.6B/glove.6B.50d.txt'

Q1_TRAINING_DATA_FILE = 'q1_train.npy'
Q2_TRAINING_DATA_FILE = 'q2_train.npy'
LABEL_TRAINING_DATA_FILE = 'label_train.npy'
WORD_EMBEDDING_MATRIX_FILE = 'word_embedding_matrix.npy'
NB_WORDS_DATA_FILE = 'nb_words.json'


Q1_TEST_DATA_FILE = 'q1_test.npy'
Q2_TEST_DATA_FILE = 'q2_test.npy'
WORD_EMBEDDING_MATRIX_FILE_TEST = 'word_embedding_matrix_test.npy'
NB_WORDS_DATA_FILE_TEST = 'nb_words_test.json'

MAX_NB_WORDS = 200000
MAX_SEQUENCE_LENGTH = 25
EMBEDDING_DIM = 50

# 학습 데이터 만들기

In [19]:
#저장하기

import os
from os.path import expanduser, exists

quora_path = expanduser('~/.kaggle/competitions/quora-question-pairs/')
quora_test = os.path.join(quora_path, 'test.csv')

df_test = pd.read_csv(quora_test, encoding='utf-8')
df_test = df_test.dropna() #drop empty

df_test

/Users/user/tf18_py_3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?
5,5,How are the two wheeler insurance from Bharti ...,I admire I am considering of buying insurance ...
6,6,How can I reduce my belly fat through a diet?,How can I reduce my lower belly fat in one month?
7,7,"By scrapping the 500 and 1000 rupee notes, how...",How will the recent move to declare 500 and 10...
8,8,What are the how best books of all time?,What are some of the military history books of...
9,9,After 12th years old boy and I had sex with a ...,Can a 14 old guy date a 12 year old girl?


In [5]:
# Tokenizer

df_train = pd.read_csv(quora_train, encoding='utf-8')
df_train = df_train.dropna() #drop empty

df_test = pd.read_csv(quora_test, encoding='utf-8')
df_test = df_test.dropna() #drop empty

questions = list(df_train.question1) + list(df_train.question2) + list(df_test.question1) + list(df_test.question2)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(questions)
word_index = tokenizer.word_index

question1_word_sequences = tokenizer.texts_to_sequences(df_train.question1)
question2_word_sequences = tokenizer.texts_to_sequences(df_train.question2)

print("Words in index: %d" % len(word_index))

question1_test_word_sequences = tokenizer.texts_to_sequences(df_test.question1)
question2_test_word_sequences = tokenizer.texts_to_sequences(df_test.question2)

print("Words in index: %d" % len(word_index))

/Users/user/tf18_py_3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Words in index: 137041
Words in index: 137041


In [7]:
# Download and extract questions pair dataset

# question1 = df_train['question1']
# question2 = df_train['question2']
is_duplicate = df_train['is_duplicate']

# with open(quora_train, encoding='utf-8') as csvfile:
#     reader = csv.DictReader(csvfile, delimiter='\t')
#     for row in reader:
#         question1.append(row['question1'])
#         question2.append(row['question2'])
#         is_duplicate.append(row['is_duplicate'])

# print('Question pairs: %d' % len(question1))

#Global embedding Load

# if not exists(quora_path  + GLOVE_ZIP_FILE):
#     zipfile = ZipFile(get_file(GLOVE_ZIP_FILE, GLOVE_ZIP_FILE_URL))
#     zipfile.extract(GLOVE_FILE, path=quora_path)
    
print("Processing", GLOVE_FILE)

embeddings_index = {}
with open(GLOVE_FILE, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings: %d' % len(embeddings_index))

#Word embedding만들기
nb_words=min(MAX_NB_WORDS, len(word_index))
word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0))


#학습 데이터
q1_data = pad_sequences(question1_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
q2_data = pad_sequences(question2_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(is_duplicate, dtype=int)
print('Shape of question1 data tensor:', q1_data.shape)
print('Shape of question2 data tensor:', q2_data.shape)
print('Shape of label tensor:', labels.shape)

#검증 데이터
q1_test_data = pad_sequences(question1_test_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
q2_test_data = pad_sequences(question2_test_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of question1 data tensor:', q1_test_data.shape)
print('Shape of question2 data tensor:', q2_test_data.shape)

Processing ./data/glove.6B/glove.6B.50d.txt
Word embeddings: 400000
Null word embeddings: 60873
Shape of question1 data tensor: (404287, 25)
Shape of question2 data tensor: (404287, 25)
Shape of label tensor: (404287,)
Shape of question1 data tensor: (3563466, 25)
Shape of question2 data tensor: (3563466, 25)


In [11]:
#학습
np.save(open(Q1_TRAINING_DATA_FILE, 'wb'), q1_data)
np.save(open(Q2_TRAINING_DATA_FILE, 'wb'), q2_data)
np.save(open(LABEL_TRAINING_DATA_FILE, 'wb'), labels)
np.save(open(WORD_EMBEDDING_MATRIX_FILE, 'wb'), word_embedding_matrix)

#검증 데이터
np.save(open(Q1_TEST_DATA_FILE, 'wb'), q1_test_data)
np.save(open(Q2_TEST_DATA_FILE, 'wb'), q2_test_data)
# np.save(open(WORD_EMBEDDING_MATRIX_FILE_TEST, 'wb'), word_embedding_matrix)
# with open(NB_WORDS_DATA_FILE_TEST, 'w') as f:
#     json.dump({'nb_words': nb_words}, f)

with open(NB_WORDS_DATA_FILE, 'w') as f:
    json.dump({'nb_words': nb_words}, f)

# Test Data 생성

In [7]:
df_test = pd.read_csv(quora_test, encoding='utf-8')
df_test = df_test.dropna() #drop empty

questions_test = list(df_test.question1) + list(df_test.question2)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(questions_test)
word_index = tokenizer.word_index

question1_test_word_sequences = tokenizer.texts_to_sequences(df_test.question1)
question2_test_word_sequences = tokenizer.texts_to_sequences(df_test.question2)

print("Words in index: %d" % len(word_index))

# Global embedding

# if not exists(quora_path  + GLOVE_ZIP_FILE):
#     zipfile = ZipFile(get_file(GLOVE_ZIP_FILE, GLOVE_ZIP_FILE_URL))
#     zipfile.extract(GLOVE_FILE, path=quora_path)
    
print("Processing", GLOVE_FILE)

embeddings_index = {}
with open(GLOVE_FILE, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings: %d' % len(embeddings_index))

/Users/user/tf18_py_3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Word embedding matrix 만들기 (For test)

In [13]:
nb_words=min(MAX_NB_WORDS, len(word_index))
word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0))

Null word embeddings: 38657


In [14]:
q1_test_data = pad_sequences(question1_test_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
q2_test_data = pad_sequences(question2_test_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of question1 data tensor:', q1_test_data.shape)
print('Shape of question2 data tensor:', q2_test_data.shape)

Shape of question1 data tensor: (3563466, 25)
Shape of question2 data tensor: (3563466, 25)


In [19]:
np.save(open(Q1_TRAINING_DATA_FILE, 'wb'), q1_data)
np.save(open(Q2_TRAINING_DATA_FILE, 'wb'), q2_data)
np.save(open(LABEL_TRAINING_DATA_FILE, 'wb'), labels)
np.save(open(WORD_EMBEDDING_MATRIX_FILE, 'wb'), word_embedding_matrix)
with open(NB_WORDS_DATA_FILE, 'w') as f:
    json.dump({'nb_words': nb_words}, f)

# 만든 데이터 저장하기 (테스트)

In [15]:
np.save(open(Q1_TEST_DATA_FILE, 'wb'), q1_test_data)
np.save(open(Q2_TEST_DATA_FILE, 'wb'), q2_test_data)
np.save(open(WORD_EMBEDDING_MATRIX_FILE_TEST, 'wb'), word_embedding_matrix)
with open(NB_WORDS_DATA_FILE_TEST, 'w') as f:
    json.dump({'nb_words': nb_words}, f)

In [5]:
# Tokenizer

df_train = pd.read_csv(quora_train, encoding='utf-8')
df_train = df_train.dropna() #drop empty

df_test = pd.read_csv(quora_test, encoding='utf-8')
df_test = df_test.dropna() #drop empty

questions_test = list(df_test.question1) + list(df_test.question2)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(questions_test)
word_index = tokenizer.word_index

question1_test_word_sequences = tokenizer.texts_to_sequences(df_test.question1)
question2_test_word_sequences = tokenizer.texts_to_sequences(df_test.question2)

print("Words in index: %d" % len(word_index))



questions = list(df_train.question1) + list(df_train.question2)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(questions)
word_index = tokenizer.word_index

question1_word_sequences = tokenizer.texts_to_sequences(df_train.question1)
question2_word_sequences = tokenizer.texts_to_sequences(df_train.question2)

print("Words in index: %d" % len(word_index))

# Download and extract questions pair dataset

# question1 = df_train['question1']
# question2 = df_train['question2']
# is_duplicate = df_train['is_duplicate']

# with open(quora_train, encoding='utf-8') as csvfile:
#     reader = csv.DictReader(csvfile, delimiter='\t')
#     for row in reader:
#         question1.append(row['question1'])
#         question2.append(row['question2'])
#         is_duplicate.append(row['is_duplicate'])

# print('Question pairs: %d' % len(question1))

#Global embedding Load

# if not exists(quora_path  + GLOVE_ZIP_FILE):
#     zipfile = ZipFile(get_file(GLOVE_ZIP_FILE, GLOVE_ZIP_FILE_URL))
#     zipfile.extract(GLOVE_FILE, path=quora_path)
    
print("Processing", GLOVE_FILE)

embeddings_index = {}
with open(GLOVE_FILE, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings: %d' % len(embeddings_index))

#Word embedding만들기
nb_words=min(MAX_NB_WORDS, len(word_index))
word_embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        word_embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0))

q1_data = pad_sequences(question1_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
q2_data = pad_sequences(question2_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(is_duplicate, dtype=int)
print('Shape of question1 data tensor:', q1_data.shape)
print('Shape of question2 data tensor:', q2_data.shape)
print('Shape of label tensor:', labels.shape)

np.save(open(Q1_TRAINING_DATA_FILE, 'wb'), q1_data)
np.save(open(Q2_TRAINING_DATA_FILE, 'wb'), q2_data)
np.save(open(LABEL_TRAINING_DATA_FILE, 'wb'), labels)
np.save(open(WORD_EMBEDDING_MATRIX_FILE, 'wb'), word_embedding_matrix)
with open(NB_WORDS_DATA_FILE, 'w') as f:
    json.dump({'nb_words': nb_words}, f)

Words in index: 95595
